In [1]:
from BetaShF import ShF
from BetaShM import ShM 
import numpy as np 
import matplotlib.pyplot as plt
import time

In [2]:
def logSample(prefix, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open("./experiments/"+name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(str(prefix) + "_Population.txt", s)
    appendToFile(str(prefix) + "_Fitness.txt", str(fitness))
    appendToFile(str(prefix) + "_ShForces.txt", str(force))
    appendToFile(str(prefix) + "_ShMoments.txt", str(moment))

#### Objective Function

In [3]:
def objective_function(s, ShF, ShM, beta): #c is a constant that distributes the weight among the functions.
    return beta*ShF(s) + (1-beta)*ShM(s)

In [4]:
# Bounds for each variable
nVar = 5
bounds = []
for i in range(1,nVar*3+1):
    if(i%3==0): bounds.append([0.005,0.04])
    else: bounds.append([-0.16, 0.16])
bounds = np.array(bounds)
print('bounds',bounds)

bounds [[-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]]


#### GD

In [5]:
#################### Descenso de Gradiente con diferenciación finita
def Gradient_Descent(X0,f,bounds,MaxIter=1000,alpha=1e-3, args = ()):
    eps = 1e-5
    def Gradiente(X,f, args = ()):
        n = len(X)
        G = np.zeros((n),float)
        incX = np.zeros((n),float)
        for i in range(n):
            incX[i] = eps
            G[i] = (f(X+incX, *args)-f(X, *args))/eps
            incX[i] = 0
        return G
    def getStepSize(alpha,m,X,P,G,f, args = ()):
        c0 = 1e-4
        c1 = 2
        c2 = 5
        c3 = 3
        eps = 1e-8
        while f(X+alpha*P, *args) > f(X, *args)+c0*np.dot(G,P):
            m = 0
            alpha = alpha/c1
            if alpha<=eps:
                break
        m += 1
        if m>=c2:
            m=0
            alpha = c3*alpha
        return alpha,m

    k=0
    X = X0
    G = Gradiente(X,f, args)
    normaGradiente = np.linalg.norm(G)
    m = 0
    while(k<=MaxIter and normaGradiente>=eps):
        G = Gradiente(X0,f, args)
        normaGradiente = np.linalg.norm(G)
        P = - G / normaGradiente
        alpha,m = getStepSize(alpha,m,X,P,G,f, args)
        X = X + alpha*P
        X = np.clip(X,bounds[:,0],bounds[:,1])
        k = k+1
        if k%100 == 0:
            print("\t\t#",k,f(X, *args))
    return X


In [6]:
def random_start(bounds):
    arr = np.zeros(bounds.shape[0])
    for i, tupl in enumerate(bounds):
        rand = np.random.uniform(tupl[0], tupl[1])
        arr[i] = rand 
    return arr
    
print(random_start(bounds))

[-0.09842577 -0.14817785  0.03179191 -0.08359883 -0.14385944  0.01968124
  0.07994792 -0.10568881  0.01791537 -0.12117131  0.09247776  0.00814046
  0.11451271 -0.0123489   0.03112376]


In [7]:
# it = 80
# n = 4
def GD(GD_alph, iter, nRuns):
    startTime_GD = str(int(time.time()))
    eTime = 0

    print(f'\n* Number of iterations: {iter}')
    for i in range(nRuns):
        r = random_start(bounds)
        beta = np.random.normal(0.5, 0.20)
        fitness = objective_function(r, ShF, ShM, beta)

        print(f"\t\nInitial Fitness: {fitness} in #{i}")
        start = time.perf_counter()
        r = Gradient_Descent(r,objective_function, bounds,MaxIter=iter, alpha=GD_alph, args=(ShF, ShM, beta))
        end = time.perf_counter()
        fit_GD = objective_function(r,ShF,ShM, beta)
        print("\t  - before: ", fitness)
        print("\t  - after (GD): ", fit_GD)
        eTime += (end-start) #Time in seconds
        shF, shM = ShF(r), ShM(r)
        if shF < 1 and shM < 1:
            logSample(startTime_GD, r, fit_GD, shF, shM) # (now, sample, fitness, force, moment):
    if nRuns: eTime /= nRuns
    print("Average time of execution:", eTime,"seconds. It was run", nRuns, "times.")
    # return fit_GD

In [8]:
it = 1000
nRuns = 100
step_size = 6e-4 # Probamos con diferentes step_size, este valor ha sido la mejor opción.

In [9]:
GD(step_size, it, nRuns)


* Number of iterations: 1000
	
Initial Fitness: 31.698880993919786 in #0
		# 100 3.8861230149945962
		# 200 3.8861230149945962
		# 300 3.8861230149945962
		# 400 3.8861230149945962
		# 500 3.8861230149945962
		# 600 3.8861230149945962
		# 700 3.8861230149945962
		# 800 3.8861230149945962
		# 900 3.8861230149945962
		# 1000 3.8861230149945962
	  - before:  31.698880993919786
	  - after (GD):  3.8861230149945962
	
Initial Fitness: 26.26530241428626 in #1
		# 100 3.20340229023938
		# 200 3.20340229023938
		# 300 3.20340229023938
		# 400 3.20340229023938
		# 500 3.20340229023938
		# 600 3.20340229023938
		# 700 3.20340229023938
		# 800 3.20340229023938
		# 900 3.20340229023938
		# 1000 3.20340229023938
	  - before:  26.26530241428626
	  - after (GD):  3.20340229023938
	
Initial Fitness: 24.932778162370326 in #2
		# 100 18.367533586322622
		# 200 38.49156267159567
		# 300 38.37334016797525
		# 400 38.300965226332636
		# 500 38.28004612725873
		# 600 38.308611126203736
		# 700 38.3883584697